In [5]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from multiprocessing import Pool
from sklearn.feature_extraction.text import CountVectorizer
from functools import reduce

In [35]:
urls = []
page_num = 1
while True:
    r = requests.get("http://devpost.com/software/popular?query=is%3Afeatured&page=" + str(page_num))
    soup = BeautifulSoup(r.text, "lxml")
    projects = soup.findAll("div", {"class": "gallery-item"})
    if len(projects) == 0:
        break
    for i in projects:
        urls.append(i.find("a")['href'])
    page_num += 1

In [119]:
def parse_user(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "lxml")
    return len(soup.findAll("div", {"gallery-item"}))

def parse_post(url):    
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "lxml")
    header = soup.find("header", {"id": "software-header"})
    title = ''
    title_tag = header.find("h1", {"id": "app-title"})
    if title is not None:
        title = title_tag.text
    subtitle = ''
    subtitle_tag = header.find("p", {"class": "large"})
    if subtitle_tag is not None:
        subtitle = subtitle_tag.text
    like_button = header.find("a", {"class": "like-button"})
    like_count = like_button.find("span", {"class": "side-count"}).text
    built_with = soup.find("div", {"id": "built-with"})
    categories = []
    if built_with is not None:
        for i in built_with.findAll("li"):
            span = i.find("span")
            if span is not None:
                categories.append(span.text)
        
    software_lh = soup.find("div", {"class": "software-list-content"})
    hackathon = ''
    winnings = []
    if software_lh is not None:
        hackathon = software_lh.find("a").text
        winnings = []
        prizes = software_lh.findAll("li")
        for i in prizes:
            winnings.append(i.text)
            
    body = soup.find("div", {"id": "app-details-left"})
    description = ''
    if body is not None:
        parts = []
        for h2, p in zip(body.findAll("h2"), body.findAll("p")):
            h2 = h2.text.replace("\n", "")
            if h2 != '':
                parts.append(h2)
            p = p.text.replace("\n", "")
            if p != '':
                parts.append(p)
        description = parts
        
    app_team = soup.find("section", {"id": "app-team"})
    team = []
    if app_team is not None:
        for i in app_team.findAll("a"):
            if i.text != '':
                team.append({"name": i.text, "url": i['href']})
        
    return {
        "title": title,
        "subtitle": subtitle,
        "like_count": like_count,
        "categories": categories,
        "hackathon": hackathon,
        "winnings": winnings,
        'description': description,
        'team': team
    }

In [75]:
a=parse_post(urls[1])

In [78]:
body=a
parts = []
for h2, p in zip(body.findAll("h2"), body.findAll("p")):
    parts.append(h2.text)
    parts.append(p.text)
descriptions = "\n".join(parts)

In [88]:
parse_post(urls[3])

{'categories': ['c++', 'python', 'svo', 'imu', 'makerbot', 'ps3-eye'],
 'description': '',
 'hackathon': 'MHacks 8',
 'like_count': '3',
 'subtitle': '\n    A personal ping pong trainer in the palm of your hand!\n  ',
 'team': [{'name': 'Helen Xu', 'url': 'http://devpost.com/hxu28602'},
  {'name': 'Mihir Garimella', 'url': 'http://devpost.com/gtmtg'},
  {'name': 'Kalyani Ramadurgam', 'url': 'http://devpost.com/kalyrama'}],
 'title': 'Stroke of Genius',
 'winnings': []}

In [120]:
p = Pool(5)
data = p.map(parse_post, urls)
    
# data = []
# for i in urls:
#     data.append(parse_post(i))
    
with open("data.json", "w") as f:
    f.write(json.dumps(data))

Process ForkPoolWorker-125:
Process ForkPoolWorker-122:
Process ForkPoolWorker-124:
Process ForkPoolWorker-123:
Process ForkPoolWorker-121:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/benawad/anaconda3/lib/python3.5/multiprocessing/process.py", line 254, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):


KeyboardInterrupt: 

Traceback (most recent call last):
  File "/Users/benawad/anaconda3/lib/python3.5/multiprocessing/process.py", line 254, in _bootstrap
    self.run()
  File "/Users/benawad/anaconda3/lib/python3.5/multiprocessing/process.py", line 254, in _bootstrap
    self.run()
  File "/Users/benawad/anaconda3/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/benawad/anaconda3/lib/python3.5/multiprocessing/process.py", line 254, in _bootstrap
    self.run()
  File "/Users/benawad/anaconda3/lib/python3.5/multiprocessing/process.py", line 254, in _bootstrap
    self.run()
  File "/Users/benawad/anaconda3/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/benawad/anaconda3/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/Users/benawad/anaconda3/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._arg

In [96]:
df = pd.read_json("./data.json")

In [14]:
def clean_winnings(s):
    s = s.replace("\n", "")
    s = s.replace("Winner", "")
    s = s.strip()
    return s

In [98]:
df['winnings'] = df['winnings'].apply(lambda x: list(map(clean_winnings, x)))
df['num_cat'] = df['categories'].apply(lambda x: len(x))
df['like_count'] = df['like_count'].astype(int)

In [29]:
def isWinner(df):
    df['winnings'] = df['winnings'].apply(lambda x: len(x) > 0)
    return df

In [30]:
df = isWinner(df)

In [102]:
df['teamsize'] = df['team'].apply(lambda x: len(x))
#df['total_hackathons_attended'] = df['team'].apply(lambda x: reduce(lambda acc, i: acc + int(i['hackathons_attended'], x, 0)))

In [103]:
df

,categories,description,hackathon,like_count,subtitle,team,title,winnings,num_cat,teamsize
0,"[microsoft-hololens, unity, c#, blender]",Inspiration\nWhat it does\nHow we built it\nWh...,"Reality, Virtually, Hackathon",10,\n Choose Your Flavor\n,"[{'url': 'http://devpost.com/yannick_boers', '...",Fizz Filter,[],4,5
1,"[leap-motion, raspberry-pi, servo, 3dprinting]",Inspiration\nYou know those games where you mo...,HackUMass IV,7,\n Move a ball through a maze without touch...,"[{'url': 'http://devpost.com/AlexDFischer', 'n...",Jedi Maze Game,[],4,2
2,[matlab],Inspiration\nImage selection\nWhat it does\nBl...,HackUPC Fall 2016,6,"\n Take the best images automaticly, in sec...","[{'url': 'http://devpost.com/HEIMDAL13', 'name...",ImaTake,[3rd Prize],1,1
3,"[c++, python, svo, imu, makerbot, ps3-eye]",Inspiration\nWhat it does\nHow we built it\nBu...,MHacks 8,3,\n A personal ping pong trainer in the palm...,"[{'url': 'http://devpost.com/hxu28602', 'name'...",Stroke of Genius,[],6,3
4,"[watson-tone-analyzer, bark.us-api, python, ja...",Inspiration\nHere is a screenshot of the chrom...,HackDartmouth III,4,"\n SurfShield is an extension for Chrome, a...","[{'url': 'http://devpost.com/AndrewOgren', 'na...",SurfShield,"[First Place, Best Hack Against Online Harassm...",14,4
5,"[arduino, 3d-print]",Inspiration\nMatt and his IntelliKnee\nWhat it...,HackGT 2016,4,\n Automatically locking knee brace - when ...,"[{'url': 'http://devpost.com/DavidGoldman', 'n...",IntelliKnee,[],2,3
6,"[javascript, jquery, python, postgresql, machi...",Inspiration\nAutomated Diabetic Retinopathy D...,Predix India Hackathon,10,\n Ultimate health care tool for predictin...,"[{'url': 'http://devpost.com/sadhanag', 'name'...",Trojan - Automated Diabetic Retinopathy Detection,[],14,1
7,"[arduino, swift, innovation, cardboard, duct-t...",Inspiration\nApp Interface\nWhat it does\nApp ...,HackCMU 2016,20,\n Have you ever been starving after classe...,"[{'url': 'http://devpost.com/kibowman', 'name'...",Bob's Ramen,"[Mentor's choice award, presented by Microsoft]",6,4
8,"[javascript, node.js, unity, c#, amazon-alexa]","Inspiration\nWhat it does\nDoctors, Students, ...",Hack the North 2016,8,\n VR Open Heart Surgery With Alexa as a Vi...,"[{'url': 'http://devpost.com/Scub3d', 'name': ...",Project Open Heart,"[Firebase API, Top 12 s]",5,2
9,"[ios, python, firebase, swift, c.h.i.p., 3dpri...",Inspiration\nWhat it does\nHow we built it\nCh...,Hack the North 2016,6,\n Self learning a string instrument is dif...,"[{'url': 'http://devpost.com/JeffreyBarton', '...",UkeMaster,[],10,3


In [105]:
tags = []
for i in df['winnings']:
    for w in i:
        for tag in w.split(' '):
            tags.append(tag.lower())

In [116]:
counter = dict(map(lambda x: (x, 0), tags))

In [117]:
for i in df['winnings']:
    for w in i:
        for tag in w.split(' '):
            t = tag.lower()
            counter[t] += 1

{'': 4,
 '$3,500': 1,
 '$4,000': 1,
 '&': 2,
 '(3': 1,
 '(best': 1,
 '(concur)': 1,
 '(general)': 2,
 '(j.p.morgan)': 1,
 '(pretty': 1,
 '(sponsored': 1,
 '(twilio)': 1,
 '(university': 1,
 '+': 1,
 '-': 22,
 '10': 3,
 '12': 1,
 '1st': 17,
 '2': 2,
 '2014': 4,
 '2015': 1,
 '2nd': 10,
 '3:': 1,
 '3d': 1,
 '3rd': 11,
 '5': 8,
 '5th': 2,
 'a': 1,
 'against': 1,
 'alexa': 1,
 'algorithm': 1,
 'all': 4,
 'alphalab': 1,
 'alumni': 1,
 'amazon': 2,
 'america': 1,
 'an': 1,
 'and': 3,
 'android': 2,
 'announced': 1,
 'api': 14,
 'apis': 3,
 'app': 7,
 'appcelerator': 1,
 'apple': 4,
 'arm': 1,
 'artistic': 1,
 'as': 1,
 'astra': 1,
 'audience': 2,
 'automotive': 1,
 'award': 8,
 'award,': 1,
 'award:': 1,
 'awesome': 1,
 'aws': 2,
 'azure': 1,
 'be': 2,
 'beginner': 1,
 'beginners': 1,
 'best': 68,
 'bloc': 1,
 'bloomberg': 1,
 'bluemix': 1,
 'built': 1,
 'business': 1,
 'but': 1,
 'by': 8,
 'cambridge': 1,
 'can': 4,
 'car': 1,
 'card': 1,
 'challenge': 4,
 'chegg:': 1,
 'choice': 8,
 'chosen